<a href="https://colab.research.google.com/github/RedaElmar/Griffin/blob/master/Execution_sur_Notebook_IA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Définition de la classe Puzzle 

In [21]:
# Définition de la classe pour un puzzle, enregistre l'état du puzzle en cours
# self.state = liste de représentation du jeu de taquin               [ 1, 2, 3
# Par exemple, le puzzle  [1, 2, 3, 4, 5, 6, 7, 8, 0] represente - >    4, 5, 6
#                                                                       7, 8, 0 ]
# Où 0 est l'case vide
#
# self.depth = la profondeur du noeud courant (initialisé à 0)
# self.prev = lien vers le nœud du puzzle parent (initialisé à None)
# cela aide à imprimer la chaîne de solutions
# self.score = score que nous utiliserons pour attribuer une valeur heuristique aux recherches informés si nécessaire
# self.solution = état de la solution du puzzle

class Puzzle:
    # Initialiser le puzzle
    def __init__(self, state, depth=0, prev=None):
        self.state = state
        self.depth = depth
        self.prev = prev
        self.score = 0
        self.solution = [1, 2, 3, 4, 5, 6, 7, 8, 0]

    # Ces fonctions sont telles que si nous comparons les objets du puzzle avec >, <, =, ils sont comparés par leur score
    def __eq__(self, value):
        return self.score == value.score

    def __gt__(self, value):
        return self.score > value.score

    def __lt__(self, value):
        return self.score < value.score

    # Renvoyer les instances de puzzles fils
    # Les mouvements valables qui pourraient être effectués sont calculés,
    # puis appliqué à l'état actuel pour créer de nouveaux objets Puzzle
    # La profondeur et le lien avec le précédent sont également fixés
    def expand(self):
        blank_idx = self.state.index(0)
        # Les déplacements possibles existent dans toutes les tuiles adjacentes (rangée au-dessus/en dessous, col à gauche et à droite)
        # Pour valider les mouvement, il suffit de vérifier si 0 <= i <= 8
        # Cependant, nous avons aussi des cas où si i est sur le bord droit, nous ne pouvons pas aller à i+1 (s'enroule autour de l'autre extrémité)
        # ou i-1 sur le bord gauche (même raison)
        #
        #  0     i-3    0
        #  i-1    i     i+1
        #  0     i+3    0

        possibleMoves = []

        # Déplacer vers le haut/vers le bas
        possibleMoves.append(blank_idx - 3)
        possibleMoves.append(blank_idx + 3)

        # Déplacer à droite/à gauche (si valable)
        if blank_idx != 2 and blank_idx != 5:
            possibleMoves.append(blank_idx + 1)
        if blank_idx != 3 and blank_idx != 6:
            possibleMoves.append(blank_idx-1)

        # Filtrer les limites du tableau
        validMoves = []
        for x in possibleMoves:
            if x >= 0 and x <= 8:
                validMoves.append(x)

        # Créer des puzzles fils avec des mouvements appliqués et mise à jour des champs
        children = []
        for idx in validMoves:
            new_state = self.state[:]
            new_state[blank_idx], new_state[idx] = new_state[idx], new_state[blank_idx]
            children.append(Puzzle(new_state, self.depth + 1, self))

        return children

    # Imprimer le puzzle
    def printPuzzle(self):
        print("*********************")
        r1 = self.state[:3]
        r2 = self.state[3:6]
        r3 = self.state[6:]
        print(str(r1))
        print(str(r2))
        print(str(r3))
        print("*********************")

    # Imprimer la chaîne de solution de ce puzzle (doit être appelée lorsqu'une solution est atteinte)
    def printSolution(self):
        puzzles = []
        while self:
            puzzles.append(self)
            self = self.prev
        puzzles.reverse()

        k = 0
        for p in puzzles:
            print("Step: ", k) if k != 0 else print("Initial State")
            p.printPuzzle()
            k += 1
        print("Solved!")


#Fonction qui vérifie la Sovabilité

In [22]:
# La solvabilité peut être déterminée en triant la liste, et en comptant le nombre de permutations nécessaires pour le trier (ignorez 0)
# Si un nombre d'échanges nécessaire est impair, le puzzle est insoluble
# Pour en avoir la preuve, voir : https://www.geeksforgeeks.org/check-instance-8-puzzle-solvable
def solvable(state):
    temp = state[:]
    temp.remove(0)
    inv = 0
    for i in range(len(temp)):
        for j in range(len(temp)-1):
            if temp[j] > temp[j+1]:
                temp[j], temp[j+1] = temp[j+1], temp[j]
                inv += 1
    return True if inv % 2 == 0 else False

#Création de la file d'attente prioritaire

In [23]:
import heapq
# Priority Queue utilisant le module python heapq (min-heap)
# L'element ayant la plus haute priorité sera maintenu en haut du tas conformément à la fonction prioritaire donnée
# self.key = fonction pour décider de la priorité (par exemple, distance de hamming)
# self.data = liste qui stockera les données (maintiendra le tas invariant avec les pops / pushes ) enfiler / défiler
class PriorityQueue:
    def __init__(self, key):
        self.key = key
        self.data = []

    def push(self, item):
        # Python heapq permet de passer un tuple, le premier élément étant la fonction de priorité,
        # et le second est le point à pousser
        # Dans notre cas, nous allons pousser les objets du puzzle qui ce nécessite une sorte de fonction de notation
        heapq.heappush(self.data, (self.key(item), item))

    def pop(self):
        # défiler l'élément ayant la plus haute priorité
        return heapq.heappop(self.data)[1]

#Définition des heuristiques: Manhattan et Hamming

In [32]:
def hamming(puzzle):
    h = 0
    for i in range(len(puzzle.state)):
        if puzzle.state[i] != puzzle.solution[i] and puzzle.state[i] != 0:
            h += 1
    return h


def manhattan(puzzle):
    score = 0
    for i in range(1, 9):
        for stateVal, goalVal in ((puzzle.state.index(i), puzzle.solution.index(i)),):
            score += (abs(stateVal % 3 - goalVal % 3) +
                      abs(stateVal // 3 - goalVal // 3))
    return score
# Utilise la priorité de distance de Manhattan Heuristique, où h distance totale des tuiles par rapport à leurs positions finales

def fscore_manhattan(puzzle):
    g = puzzle.depth
    h = manhattan(puzzle)
    return h + g


# Utilise la priorité de Hamming Heuristic, où h est le nombre de tuiles mal placées (sans compter les espaces vides)

def fscore_hamming(puzzle):
    g = puzzle.depth
    h = hamming(puzzle)
    return h + g

# puzzle = puzzle à résoudre
# score_func = fonction de score (peut utiliser manhattan ou hamming priorities si désiré)


#BFS

In [25]:
# Le BFS offre une solution optimale
# L'algorithme fonctionne en parcourant les nœuds dans l'ordre de 
# leur niveau (combien de mouvement ont été appliqués)
# Cela garantit une solution optimale en raison de la traversée de l'ordre des niveaux,
# nous ne vérifierons jamais plus profondément avant que tous les nœuds
# d'un niveau actuel ont été explorés

def BFS(puzzle):
    queue = [puzzle]                                # Initialiser la file en tant que liste python
    visited = {}                                    # Initialiser le dictionnaire pour 
                                                    # garder une trace des états visités

    while(queue):                                                   
        cur = queue.pop(0)                          # Ouvrir et retirer (pop) le nœud actuel 
                                                    # (ce sera un niveau supérieur à celui des enfants)
        if cur.state == cur.solution:               # verifier si c'est la solution
            cur.printSolution()
            return

        visited[str(cur.state)] = True              # Marquer l'état comme visité 
                                                    # et ajouter les fils
        children = cur.expand()                     # (appliquer les éventuels mouvements)

        for child in children:                      # Ajouter les fils à la file 
            if str(child.state) not in visited:     # s'ils n'ont pas déjà été explorés 
                queue.append(child) 
                                                    # Tous les états sont épuisés
    print("No Solution")                            # pas de solutions pour ce puzzle

#Greedy

In [27]:
# Cet algorithme ne garantit pas une solution optimale
# Cette recherche utilise simplement une valeur heuristique
# pour choisir l'enfant à développer
# En choisissant toujours le nœud le plus proche de la solution,
# on arrive assez vite à la solution, mais de façon non optimale
# en termes de mouvements effectués

def GREEDY(puzzle, heuristic=hamming):
    puzzle.score = heuristic(puzzle)                    # Définir le score initial en utilisant l'heuristique 
    queue = PriorityQueue(heuristic)                    # Initialiser la file avec l'heuristique 
                                                        # donnée pour la priorité 
    queue.push(puzzle)                                  # Ajouter le premier puzzle à la file 
    visited = {}                                        # Initialiser le dictionnaire pour garder 
                                                        # une trace des états visités

    while queue:                                        
        cur = queue.pop()                               # Ouvrir et retirer(pop) l'élément avec 
                                                        # le meilleur score heuristique (situé à l'avant) 
        visited[str(cur.state)] = True                  # Marquez l'état actuel comme visité 
        if cur.state == cur.solution:                   # Vérifiez la solution, si oui, quittez
            cur.printSolution()
            return

        children = cur.expand()                         # Développer le nœud actuel, attribuer 
                                                        # des scores et inserer dans la file.
        for child in children:                          # La file d'attente conservera le meilleur 
                                                        # puzzle en haut pour qu'il soit ouvert.
            child.score = heuristic(child)
            if str(child.state) not in visited:
                queue.push(child)

    print("No Solution")                                # Aucune solution trouvée (puzzle insoluble)

#A Star

In [26]:
# L'algorithme A* donne à chaque État un score f = g + h où
# g = est le coût du mouvement (nombre de mouvements pour atteindre l'état, dans ce cas c'est la profondeur du puzzle)
# h = est le coût estimé du mouvement / heuristique (nous pouvons utiliser la distance de Manhattan 
# ou une tuile mal placée pour obtenir h)

# Nous choisissons alors de traverser le nœud ayant le score f le plus bas, ce qui, espérons-le, 
# nous permet d'obtenir le chemin le moins coûteux
# A tout moment, nous gardons une liste ouverte et fermée

def A_STAR(puzzle, score_func=fscore_manhattan):
    puzzle.score = score_func(puzzle)             # Définir le score initial en utilisant l'heuristique 
    queue = PriorityQueue(score_func)             # la File de Priorité  conserve les nœuds 
    queue.push(puzzle)                            # que nous devons encore explorer
    visited = {}
    while queue:
        cur = queue.pop()
        visited[str(cur.state)] = True
        if cur.state == cur.solution:
            cur.printSolution()
            return
        children = cur.expand()
        for child in children:                    # visited conserve les nœuds 
            child.score = score_func(child)       # que nous avons deja explorés
            if str(child.state) not in visited:   # nous vérifions avant d'ajouter un noeud enfant 
                queue.push(child)                 # (nous ne voulons pas recalculer le même état)

#Donnes de test et fonction d'affichage

In [ ]:
import time
tests = [
    #Puzzle([1, 2, 3, 4, 5, 6, 7, 0, 8]), #Cas Trivial
    #Puzzle([2, 4, 3, 1, 0, 5, 7, 8, 6]), #Cas facile
    Puzzle([3, 6, 5, 7, 1, 8, 0, 2, 4]),  # Cas normale
    #Puzzle([0, 3, 5, 1, 6, 8, 7, 2, 4]), # Cas normale
    #Puzzle([8, 7, 6, 5, 4, 3, 2, 1, 0]), # Pire des Cas
    #Puzzle([8, 1, 2, 0, 4, 3, 7, 6, 5]), # Unsolvable
]

def run_test(search_alg, times, name, testNum):
    if not solvable(tests[testNum].state):
        print("Test: ", testNum+1, " is unsolvable.")
        print("\n\n")
        return

    print("\n\n")
    print("Running Test: ", testNum+1, " with ", name)
    print("-------------------------------------------------------------------")
    start = time.time()
    search_alg(tests[testNum])
    diff = time.time()-start
    times.append(diff)
    print("Test: ", testNum+1, " Finished in: ", round(diff, 4))
    print("-------------------------------------------------------------------")
    print("\n\n")

bfs_times = []
greedy_times = []
a_star_times = []



#Test de BFS

In [34]:
for i in range(len(tests)):
  run_test(BFS, bfs_times, "BFS", i)




Running Test:  1  with  BFS
-------------------------------------------------------------------
Initial State
*********************
[3, 6, 5]
[7, 1, 8]
[0, 2, 4]
*********************
Step:  1
*********************
[3, 6, 5]
[0, 1, 8]
[7, 2, 4]
*********************
Step:  2
*********************
[3, 6, 5]
[1, 0, 8]
[7, 2, 4]
*********************
Step:  3
*********************
[3, 0, 5]
[1, 6, 8]
[7, 2, 4]
*********************
Step:  4
*********************
[0, 3, 5]
[1, 6, 8]
[7, 2, 4]
*********************
Step:  5
*********************
[1, 3, 5]
[0, 6, 8]
[7, 2, 4]
*********************
Step:  6
*********************
[1, 3, 5]
[7, 6, 8]
[0, 2, 4]
*********************
Step:  7
*********************
[1, 3, 5]
[7, 6, 8]
[2, 0, 4]
*********************
Step:  8
*********************
[1, 3, 5]
[7, 6, 8]
[2, 4, 0]
*********************
Step:  9
*********************
[1, 3, 5]
[7, 6, 0]
[2, 4, 8]
*********************
Step:  10
*********************
[1, 3, 5]
[7, 0, 6]
[2, 4, 8]
****

#Test de GREEDY

In [29]:
for i in range(len(tests)):
  run_test(GREEDY, greedy_times, "GREEDY", i)





Running Test:  1  with  GREEDY
-------------------------------------------------------------------
Initial State
*********************
[3, 6, 5]
[7, 1, 8]
[0, 2, 4]
*********************
Step:  1
*********************
[3, 6, 5]
[0, 1, 8]
[7, 2, 4]
*********************
Step:  2
*********************
[3, 6, 5]
[1, 0, 8]
[7, 2, 4]
*********************
Step:  3
*********************
[3, 0, 5]
[1, 6, 8]
[7, 2, 4]
*********************
Step:  4
*********************
[0, 3, 5]
[1, 6, 8]
[7, 2, 4]
*********************
Step:  5
*********************
[1, 3, 5]
[0, 6, 8]
[7, 2, 4]
*********************
Step:  6
*********************
[1, 3, 5]
[6, 0, 8]
[7, 2, 4]
*********************
Step:  7
*********************
[1, 3, 5]
[6, 8, 0]
[7, 2, 4]
*********************
Step:  8
*********************
[1, 3, 5]
[6, 8, 4]
[7, 2, 0]
*********************
Step:  9
*********************
[1, 3, 5]
[6, 8, 4]
[7, 0, 2]
*********************
Step:  10
*********************
[1, 3, 5]
[6, 0, 4]
[7, 8, 2]
*

#Test de A Star

In [33]:
for i in range(len(tests)):
  run_test(A_STAR, a_star_times, "A*", i)




Running Test:  1  with  A*
-------------------------------------------------------------------
Initial State
*********************
[3, 6, 5]
[7, 1, 8]
[0, 2, 4]
*********************
Step:  1
*********************
[3, 6, 5]
[7, 1, 8]
[2, 0, 4]
*********************
Step:  2
*********************
[3, 6, 5]
[7, 1, 8]
[2, 4, 0]
*********************
Step:  3
*********************
[3, 6, 5]
[7, 1, 0]
[2, 4, 8]
*********************
Step:  4
*********************
[3, 6, 0]
[7, 1, 5]
[2, 4, 8]
*********************
Step:  5
*********************
[3, 0, 6]
[7, 1, 5]
[2, 4, 8]
*********************
Step:  6
*********************
[3, 1, 6]
[7, 0, 5]
[2, 4, 8]
*********************
Step:  7
*********************
[3, 1, 6]
[0, 7, 5]
[2, 4, 8]
*********************
Step:  8
*********************
[3, 1, 6]
[2, 7, 5]
[0, 4, 8]
*********************
Step:  9
*********************
[3, 1, 6]
[2, 7, 5]
[4, 0, 8]
*********************
Step:  10
*********************
[3, 1, 6]
[2, 0, 5]
[4, 7, 8]
*****

In [40]:
!jupyter nbconvert --to PDF "Execution_sur_Notebook_IA.ipynb"

[NbConvertApp] Converting notebook Execution_sur_Notebook_IA.ipynb to PDF
[NbConvertApp] Writing 61425 bytes to ./notebook.tex
[NbConvertApp] Building PDF
[NbConvertApp] Running xelatex 3 times: [u'xelatex', u'./notebook.tex', '-quiet']
[NbConvertApp] Running bibtex 1 time: [u'bibtex', u'./notebook']
[NbConvertApp] WARNING | bibtex had problems, most likely because there were no citations
[NbConvertApp] PDF successfully created
[NbConvertApp] Writing 60517 bytes to Execution_sur_Notebook_IA.pdf
